<a href="https://colab.research.google.com/github/waseGithub/barrels/blob/master/data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import shutil
import os
from pathlib import Path
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
from google.colab import data_table
data_table.enable_dataframe_formatter()


Mounted at /content/gdrive


In [9]:
# def get_csv(path_loc):
#   '''Takes a path to csv folder, iterates over the folder
#       adjuststing the tank ID to an integer value '''
#   path2csv = Path(path_loc)
#   csvlist = path2csv.glob("*.csv")
#   ls = []
#   df = pd.DataFrame()
#   for csv in csvlist:
#     df = pd.read_csv(csv)
#     df['ID'] = pd.to_numeric(df['ID'], errors='coerce')
#     df = df.fillna(0)
#     df['ID'] = df['ID'].astype('int')  
#     ls .append(df)
  
#   df = pd.concat(ls, axis=0)  
#   df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
#   df.set_index(['ID', 'datetime'], inplace = True)
#   df = df.drop(columns=['date'])
#   df = df.apply(pd.to_numeric, errors='coerce')
  
#   return df


#load csv 
#convert to in 
#convert to datetime 
#set universal index 

def get_csv(path_loc):
  '''Takes a path to csv folder, iterates over the folder
      adjuststing the tank ID to an integer value '''
  path2csv = Path(path_loc)
  csvlist = path2csv.glob("*.csv")
  ls = []
  df = pd.DataFrame()
  for csv in csvlist:
    df = pd.read_csv(csv)
    ls.append(df)
  return ls 

def list_to_df(ls):
  df = pd.concat(ls, axis=0)
  return df 

def WASE_universal_index(df):
  df['ID'] = pd.to_numeric(df['ID'], errors='coerce')
  df = df.fillna(0)
  df['ID'] = df['ID'].astype('int')  
  df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
  df.set_index(['ID', 'datetime'], inplace = True)
  df = df.drop(columns=['date'])
  df = df.apply(pd.to_numeric, errors='coerce')
  return df



def resample_mean(df, time, cols, round_val):
  df= df[(df >= 0.0).all(1)]
  df = df.groupby([pd.Grouper(freq=time, level='datetime'), pd.Grouper(level='ID')])[cols].mean() 
  df = df.round(round_val)
  return df

def resample_sum(df, time, cols, round_val):
  df= df[(df >= 0.0).all(1)]
  df = df.groupby([pd.Grouper(freq=time, level='datetime'), pd.Grouper(level='ID')])[cols].sum()
  df = df.round(round_val)
  return df

def resample_max(df, time, cols, round_val):
  df = df.fillna(0)
  df = df.round(round_val)
  df = df.groupby([pd.Grouper(freq=time, level='datetime'), pd.Grouper(level='ID')])[cols].max() 
  return df

def make_df(df_gas, df_power, time):
  df_temp = resample_mean(df_gas, time, ['Tmp1', 'Tmp2', 'TmpAV'], 1)
  df_gas1 = resample_max(df_gas, time, ['Cnt'], 1)
  df_gas2 = resample_sum(df_gas, time, ['Cnt_delta', 'volume'], 1)
  df_power = resample_mean(df_power, time, ['V', 'A', 'P'], 2)
  df_ls = [df_temp, df_gas1 ,df_gas2, df_power]
  df_out = pd.concat(df_ls, axis=1)
  return df_out 


def plot_timeseries(df, column, title, y_axis):
  fig = go.Figure()
  #  df[column] = df[column][(df[[column]] > 0).all(1)]
  #  df[column] = df[column][(df[[column]] < 30000).all(1)]
  try:
    df.reset_index(inplace = True)
  except ValueError:
    pass

  for i in np.arange(1,5,1):

    fig.add_trace(go.Scatter(
        x=df[df['ID'] == i]['datetime'],
        y=df[df['ID'] == i][column],
        name = 'Tank' + str(i),
        line_shape='spline',
        connectgaps= False
    # Style name/legend entry with html tags
    ))

  fig.update_layout(
      title={
          'text': str(title) + " barrels",
          'y':0.9,
          'x':0.08,
          'xanchor': 'left',
          'yanchor': 'top'})

  fig.update_yaxes(showline=True, linewidth=2, gridcolor='lightgray', linecolor='black')
  fig.update_xaxes(showline=True, linewidth=2, gridcolor='lightgray', linecolor='black')
  fig.update_layout(legend=dict(orientation="v",yanchor="top",y=0.99,xanchor="right",x=1.01))
  fig.update_layout(template = "plotly_white", width=1000, height=600, yaxis_title=y_axis,font=dict(size=15), margin=dict(l=50,r=50,b=50,t=50,pad=3))

  fig.update_layout(
    xaxis=dict(
        rangeselector=dict(y=1.1,
            buttons=list([
                dict(count=1,
                     label="1d",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="1w",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    )
)



  fig.show()



In [10]:
pathls = ["/content/gdrive/MyDrive/Barrels/cleaned_tip_temp", "/content/gdrive/MyDrive/Barrels/cleaned_power"]





power_ls = get_csv(pathls[1])
gas_ls = get_csv(pathls[0])

df_power = list_to_df(power_ls)
df_gas = list_to_df(gas_ls)

df_power = WASE_universal_index(df_power)
df_gas = WASE_universal_index(df_gas)






df_gas['TmpAV'] = (df_gas['Tmp1'] + df_gas['Tmp2'])/2


df_1H = make_df(df_gas, df_power, '60T')
df_1D = make_df(df_gas, df_power, '1D')



# data_table.DataTable(df_gas.tail(100), num_rows_per_page=25)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning:

Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.



In [11]:
data_table.DataTable(df_1H.tail(100), num_rows_per_page=25)

In [12]:
data_table.DataTable(df_1D.tail(25), num_rows_per_page=25)

In [13]:
plot_timeseries(df_1H,'volume', 'Volumetric flow', 'Volumetric Flow Rate (ml/hr)')
plot_timeseries(df_1H, 'A', 'Current', 'Module current (A)')
plot_timeseries(df_1H, 'V', 'Voltage', 'Module Voltage (A)')
plot_timeseries(df_1H, 'TmpAV', 'Average Temperature', 'Barrel Temperature (' + u'\N{DEGREE SIGN}' + 'C)')

In [14]:
plot_timeseries(df_1D,'volume', 'Volumetric flow', 'Volumetric Flow Rate (ml/hr)')
plot_timeseries(df_1D, 'A', 'Current', 'Module current (A)')
plot_timeseries(df_1D, 'TmpAV', 'Average Temperature', 'Barrel Temperature (' + u'\N{DEGREE SIGN}' + 'C)') 